In [51]:
from game import Game
import time
import os
import re

## Load maps

In [52]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [53]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [54]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [55]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [56]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [57]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [58]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [59]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

### BFS

In [60]:
from collections import deque

def solver_bfs(map):
    game = Game(map)
    start_time = time.time()
    queue = deque([(game.get_player_position(), tuple(game.get_box_locations()), "")])
    visited = set()
    
    while queue:
        if(time.time() - start_time > 60):
            return None, -1
        player_position, boxes_position, path = queue.popleft()
        state_hash = (player_position, boxes_position)
        
        if state_hash in visited:
            continue
        visited.add(state_hash)


        for move in ['U', 'D', 'L', 'R']:
            game.set_player_position(player_position)
            game.set_box_positions(boxes_position)

            if game.apply_move(move):
                if game.is_game_won():
                    return path + move, len(visited)
                new_state = (game.get_player_position(), tuple(game.get_box_locations()), path + move)
                queue.append(new_state)
                
    
    return None, 0

### DFS

In [61]:
def solver_dfs(map, depth_limit=1000):
    game = Game(map)
    start_time = time.time()
    stack = [(game.get_player_position(), tuple(game.get_box_locations()), "")]
    visited = set()

    while stack:
        if(time.time() - start_time > 60):
            return None, -1
        
        player_position, boxes_position, path = stack.pop()
        state_hash = (player_position, boxes_position)

        if state_hash in visited:
            continue

        visited.add(state_hash)

        if len(path) >= depth_limit:
            continue

        for move in ['U', 'D', 'L', 'R']:
            game.set_player_position(player_position)
            game.set_box_positions(boxes_position)

            if game.is_game_won():
                    return path, len(visited)
            
            if game.apply_move(move):
                if game.is_game_won():
                    return path + move, len(visited)
                new_state = (game.get_player_position(), tuple(game.get_box_locations()), path + move)  
                stack.append(new_state)
    
    return None, 0

### IDS

In [62]:
def solver_ids(game_map, max_depth=1000):
    game = Game(game_map)
    start_time = time.time()
    
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    total_visited = 0
    
    for current_depth in range(max_depth):
        path_result, visit_count = bfs_with_depth_limit(game, initial_state, current_depth, start_time)
        total_visited += visit_count
        
        if path_result is not None or visit_count < 0:
            return path_result, total_visited
    
    return None, total_visited

def bfs_with_depth_limit(game, initial_state, depth_limit, start_time):
    visited = set()
    queue = deque([(initial_state[0], initial_state[1], "")])
    visit_count = 0
    
    while queue:
        if time.time() - start_time > 60:
            return None, -1
            
        player_pos, boxes_pos, path = queue.popleft()
        
        if (player_pos, boxes_pos) in visited:
            continue
            
        visited.add((player_pos, boxes_pos))
        visit_count += 1

        game.set_player_position(player_pos)
        game.set_box_positions(boxes_pos)
        if game.is_game_won():
            return path, visit_count
            
        if len(path) >= depth_limit:
            continue
            
        for move in ['U', 'D', 'L', 'R']:
            game.set_player_position(player_pos)
            game.set_box_positions(boxes_pos)
            
            if game.apply_move(move):
                new_state = (
                    game.get_player_position(),
                    tuple(game.get_box_locations()),
                    path + move
                )
                queue.append(new_state)
    
    return None, visit_count

### A*

In [63]:
import heapq
import time

def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def heuristic(game):
    boxes = game.get_box_locations()
    goals = game.get_goal_locations()
    player_pos = game.get_player_position()
    
    total_distance = 0
    boxes_not_on_goal = []
    
    for i in range(len(boxes)):
        box_pos = boxes[i]
        goal_pos = goals[i]
        box_to_goal = manhattan_distance(box_pos, goal_pos)
        total_distance += box_to_goal
        
        if box_pos != goal_pos:
            boxes_not_on_goal.append(box_pos)
    
    if boxes_not_on_goal:
        min_player_distance = min(manhattan_distance(player_pos, box) for box in boxes_not_on_goal)
        min_adj_distance = max(0, min_player_distance - 1)
        total_distance += min_adj_distance
    
    return total_distance

def solver_astar(game_map, heuristic=heuristic):
    from game import Game
    
    TIME_LIMIT = 10
    
    game = Game(game_map)
    start_time = time.time()
    start_boxes = tuple(game.get_box_locations())
    start_player = game.get_player_position()
    start_state = (start_player, start_boxes, "")
    
    heap = []
    heapq.heappush(heap, (heuristic(game), 0, start_state))
    visited = set()
    
    while heap:
        if time.time() - start_time > TIME_LIMIT:
            return None, -1
        
        current_f, current_g, current_state = heapq.heappop(heap)
        player_pos, boxes_pos, path = current_state
        state_hash = (player_pos, boxes_pos)
        
        if state_hash in visited:
            continue
        visited.add(state_hash)
        
        game.set_player_position(player_pos)
        game.set_box_positions(list(boxes_pos))
        
        if game.is_game_won():
            return path, len(visited)
        
        for move in ['U', 'D', 'L', 'R']:
            if game.apply_move(move):
                new_player = game.get_player_position()
                new_boxes = tuple(game.get_box_locations())
                new_path = path + move
                new_g = current_g + 1
                new_h = heuristic(game)
                new_f = new_g + new_h
                
                heapq.heappush(heap, (new_f, new_g, (new_player, new_boxes, new_path)))
                
                game.set_player_position(player_pos)
                game.set_box_positions(list(boxes_pos))
    
    return None, 0

In [64]:
import heapq
import time

def solver_weighted_astar(game_map, heuristic=heuristic, weight=5.0):
    from game import Game

    game = Game(game_map)
    start_time = time.time()
    start_boxes = tuple(game.get_box_locations())
    start_player = game.get_player_position()
    start_state = (start_player, start_boxes, "")
    
    heap = []
    initial_h = heuristic(game)
    heapq.heappush(heap, (weight * initial_h, 0, start_state))
    visited = set()
    
    while heap:
        if time.time() - start_time > 60:
            return None, -1
        
        current_f, current_g, current_state = heapq.heappop(heap)
        player_pos, boxes_pos, path = current_state
        state_hash = (player_pos, boxes_pos)
        
        if state_hash in visited:
            continue
        visited.add(state_hash)
        game.set_player_position(player_pos)
        game.set_box_positions(list(boxes_pos))
        if game.is_game_won():
            return path, len(visited)
        
        for move in ['U', 'D', 'L', 'R']:
            if game.apply_move(move):
                new_player = game.get_player_position()
                new_boxes = tuple(game.get_box_locations())
                new_path = path + move
                new_g = current_g + 1
                new_h = heuristic(game)
                new_f = new_g + weight * new_h
                
                heapq.heappush(heap, (new_f, new_g, (new_player, new_boxes, new_path)))
                
                game.set_player_position(player_pos)
                game.set_box_positions(list(boxes_pos))
    
    return None, 0

## Solve

In [65]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "Weighted A*": solver_weighted_astar
}

In [66]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [67]:
solve(1, "BFS")

BFS took 0.0 seconds on map 1 and visited 40 states.
7 moves were used: UDDULRR


In [68]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            solve(map, method)
            

In [69]:
solve_all()

BFS took 0.0 seconds on map 1 and visited 40 states.
7 moves were used: UDDULRR
DFS took 0.0 seconds on map 1 and visited 8 states.
7 moves were used: RLLRDUU
IDS took 0.0 seconds on map 1 and visited 189 states.
7 moves were used: UDDULRR
A* took 0.0 seconds on map 1 and visited 44 states.
7 moves were used: DULRRLU
Weighted A* took 0.0 seconds on map 1 and visited 8 states.
7 moves were used: UDLRRLD
BFS took 0.0 seconds on map 2 and visited 18 states.
6 moves were used: LUDDUL
DFS took 0.0 seconds on map 2 and visited 7 states.
6 moves were used: LLRDUU
IDS took 0.0 seconds on map 2 and visited 74 states.
6 moves were used: LUDDUL
A* took 0.0 seconds on map 2 and visited 18 states.
6 moves were used: LDULRU
Weighted A* took 0.0 seconds on map 2 and visited 7 states.
6 moves were used: LUDLRD
BFS took 0.0 seconds on map 3 and visited 111 states.
13 moves were used: ULDDUUUURDDDD
DFS took 0.0 seconds on map 3 and visited 42 states.
13 moves were used: ULDDUUUURDDDD
IDS took 0.0 second


15 moves were used: ULDDRDLLLUUURUL
DFS took 0.02 seconds on map 5 and visited 3058 states.
187 moves were used: LRDLLLLUURRDRRDLLRRDLLLLURRLLDRRRRUULLLRRRDLRDLLLLURRLLDRRRURDLLLLURRULURRRDLRDLLLLURRLLDRRRRULRDLLURRDLLLLURURLDRRRDLLLLUURRDRRULRDLLLLURURDRRDLLRRDLLLLURRLLDRRRRUULDRDLRDLLLLURURDRRDLLL
IDS took 0.19 seconds on map 5 and visited 26044 states.
15 moves were used: ULDDRDLLLUUURUL
A* took 0.01 seconds on map 5 and visited 520 states.
15 moves were used: LULDDRDLLUUURUL
Weighted A* took 0.0 seconds on map 5 and visited 84 states.
15 moves were used: LULDLRDRDLLULUU
BFS took 0.19 seconds on map 6 and visited 15082 states.
34 moves were used: UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR
DFS took 0.43 seconds on map 6 and visited 36639 states.
961 moves were used: RRRRDLLLLLLLLLDRRRRRRRRRDLLLLLLLLLDRRRRRRRRRDLRULLLLLLLLLDRRRRLLLLURRRRRRRRRULLLLLLLLLURRRRRRRRRULLLLLLLLLURRRRRRRRRULLLLLLLLLURRRRRRRRRULLLLLLLLLURRRRRRRRRULRDLLLLLLLLLDRRRRRRRRRDLLLLLLLLLDRRRRRRRRRDLLLLLLLLLDRRRRRRRRRDLLLLLLLL